In [2]:
# Solution: [[0,1,2],[3,4,5],[6,7,8]]
class PuzzleNode:
    
    def __init__(self, state, f, g, parent = None):
        """State is the full grid defining the node.
        h_cost is the heuristic cost computed with the heuristic function in the A* solver
        """
        self.state = state
        self.f = f
        self.g = g
        self.parent = parent
        self.pruned = False
        
    
    def coord_blank(self):
    #the solver will use this to add states to the frontier
        for y in range(len(self.state)):
            for x in range(len(self.state[y])):
                if self.state[y][x] == 0: return y, x    
               
    
    def pos_blank(self):
        # Get current position of blank tile
        for n, row in enumerate(self.state): 
            if 0 in row: 
                blank_y = n
                break
        blank_x = self.state[blank_y].index(0)
        return blank_y, blank_x    
    
 
    # Comparison function based on f cost
    def __lt__(self,other):
        return self.f < other.f

    def __str__(self):
        return "\n".join([" ".join([str(x) for x in row]) for row in self.state])    
                                               

In [3]:
## Heuristics

#uniform cost search behavior
def hnull(state): return 0


## Count number of misplaced tiles
def misplaced_tiles(state):
    n = len(state)
    val = 0
    for row_ind in range(n):
        for col_ind in range(n):
            if state[row_ind][col_ind] != row_ind*n + col_ind: val+=1
    return val

#Manhattan distance for every tile
#On every move we move two tiles. In the best case scenario, this move gets both tiles closer to their destination.
#Hence, we will take the manhattan distance for each tile, and divide it by 2.

def manhattan_distance(state, goal = None):
    n = len(state)
    dist = 0    
    for row_ind, row in enumerate(state):
        for col_ind, item in enumerate(row):            
            row_ind_goal = int(item/n)
            col_ind_goal = item%n
            dist += abs(row_ind_goal - row_ind) + abs(col_ind_goal - col_ind)
    return dist / 2.0

heuristics = [misplaced_tiles, manhattan_distance,  hnull]

In [6]:
from queue import PriorityQueue

def solvePuzzle(n, state, heuristic, prnt = False):
    """
    INPUT ARGUMENTS
    —n - the puzzle dimension (i.e. n x n board)
    —state - the starting (scrambled) state of the puzzle, provided as a list of lists, with the blank space represented by the number 0. For example, for n=3, we could have state =[[7 2 4],[5 0 6],[8 3 1]] as in the image shown previously.
    —heuristic - a handle to a heuristic function (more details in the next step)
    —prnt - a boolean value that indicates whether or not to print the solution
    OUTPUT ARGUMENTS
    —steps - the number of steps required to reach the goal state from the initial state
    —frontierSize - the maximum size of the frontier during the search
    —err - an error code (-1 for bad input size or numbers)
    """
    
    #Check dimensions and numbers from 0 to n exactly once
    #For duplicates, we can simply check if every number from 0 to n appears (if they are appear and size is right,
    # then there are no duplicates). 
    if (len(state) != n or not all([len(state[i]) == n for i in range(n)])  
        or not all([any(i in row for row in state) for i in range(n)])): return 0,0,-1
    
    
    start_node = PuzzleNode(state, heuristic(state), 0)
    
    
    #Define goal state (assuming grid is always square and solution is to write numbers in ascending order)
    goal = [[row_ind*n + col_ind for col_ind in range(n)] for row_ind in range(n)]
    
    # Dictionary with current cost to reach all visited nodes
    costs_db = {str(state): start_node}

    # Frontier, stored as a Priority Queue to maintain ordering
    frontier = PriorityQueue()
    frontier.put(start_node)

    # Begin A* Search
    expansion_counter = 0

    while not frontier.empty():
        # Take the next available node from the priority queue
        cur_node = frontier.get()

        if cur_node.pruned: continue # Skip if this node has been marked for removal
            
        # Check if we are at the goal
        if cur_node.state == goal: 
            print "goal"
            break
            
        #Get coordinates of blank (0) tile:
        y_blank, x_blank = cur_node.coord_blank()
        
        #at most 4 possible swaps
        for dx, dy in [(-1,0),(1,0),(0,1),(0,-1)]:
            if y_blank + dy > 0 and y_blank + dy < n and x_blank + dx > 0 and x_blank + dx < n:
               
                '''
                #swap tiles for the new state
                m = cur_node.state[y_blank + dy][x_blank + dx]
                next_state = [row[:] for row in cur_node.state]
                next_state[y_blank + dy][x_blank + dx] = 0
                next_state[y_blank][x_blank] = m
                '''
  
                cache = cur_node.state[y_blank + dy][x_blank + dx]
                next_state = [row[:] for row in cur_node.state] # deep copy
                next_state[y_blank + dy][x_blank + dx] = 0
                next_state[y_blank][x_blank] = cache

                
                #cost to get there from this parent
                g = cur_node.g + 1
                
                #have we seen this state before? If yes, how long is the shortest path there?
                if str(next_state) in costs_db:
                    if costs_db[str(next_state)].g > g:
                        costs_db[str(next_state)].pruned = True # Mark existing value for deletion from frontier
                    else:
                        continue # ignore this child, since a better path has already been found previously.            
            
                h = heuristic(next_state) # Heuristic cost from next node to goal
                next_node = PuzzleNode(next_state, g + h, g, cur_node) # Create new node for child
                frontier.put(next_node)
                costs_db[str(next_state)] = next_node #Mark the node as explored    
        expansion_counter = expansion_counter + 1

        
    if prnt:
        optimal_path = [cur_node.state]
        while cur_node.parent:
            optimal_path.append((cur_node.parent).state)
            cur_node = cur_node.parent
        print("A* search completed in %d steps\n"% expansion_counter)
        print("Optimal Path to Goal:\n")
        for state in optimal_path[::-1]:
            print(state)
        print("Current frontier size: %d"%frontier.qsize())            
        
    return expansion_counter, frontier.qsize(), 0

In [8]:
import time

start = time.time()
for state in [[[5,7,6],[2,4,3],[8,1,0]], [[7,0,8],[4,6,1],[5,3,2]], 
              [[2,3,7],[1,8,0],[6,5,4]]]:
    print(state)
    for heuristic in heuristics:
        print(solvePuzzle(len(state[0]), state, heuristic, False))
print(time.time() - start)

[[5, 7, 6], [2, 4, 3], [8, 1, 0]]
(12, 0, 0)
(12, 0, 0)
(12, 0, 0)
[[7, 0, 8], [4, 6, 1], [5, 3, 2]]
(13, 0, 0)
(13, 0, 0)
(13, 0, 0)
[[2, 3, 7], [1, 8, 0], [6, 5, 4]]
(12, 0, 0)
(12, 0, 0)
(12, 0, 0)
0.00715279579163


In [7]:
state = [[5,7,6],[2,4,3],[8,1,0]]
print(solvePuzzle(len(state[0]), state, misplaced_tiles, True))

A* search completed in 12 steps

Optimal Path to Goal:

[[5, 7, 6], [2, 4, 3], [8, 1, 0]]
[[5, 7, 6], [2, 4, 3], [8, 0, 1]]
[[5, 7, 6], [2, 0, 3], [8, 4, 1]]
[[5, 7, 6], [2, 3, 0], [8, 4, 1]]
[[5, 7, 6], [2, 3, 1], [8, 4, 0]]
[[5, 7, 6], [2, 3, 1], [8, 0, 4]]
[[5, 7, 6], [2, 0, 1], [8, 3, 4]]
Current frontier size: 0
(12, 0, 0)


In [40]:
state = [[7,0,8],[4,6,1],[5,3,2]]
print(solvePuzzle(len(state[0]), state, misplaced_tiles, True))

A* search completed in 20 steps

Optimal Path to Goal:

[[7, 0, 8], [4, 6, 1], [5, 3, 2]]
[[7, 6, 8], [4, 0, 1], [5, 3, 2]]
[[7, 8, 0], [4, 0, 1], [5, 3, 2]]
[[7, 0, 0], [4, 0, 1], [5, 3, 2]]
Current frontier size: 0
(20, 0, 0)
